In [181]:
import pandas as pd
import numpy as np

import re
from __future__ import division

In [182]:
import requests
RESULTS = list();
def evaluate(csv):
    url = "http://gorilla.bigdama.tu-berlin.de:8001/DataCleaningEvaluator/"
    r = requests.post(url, data={"value": csv})
    if r.status_code != 200:
            print "There is an error! The error code:", r.status_code
    print r.text
    RESULTS.append("  ".join(r.text.split("\n")[4:7]))
    return r.text


# Task 1
## Error Detection

Here, we want to only detect data errors. To mark a cell as data error, you just need to
change its value into something else.
- Which of the mentioned data quality constraints can help you to detect data errors?
How?
- Report your best error detection precision, recall, and F1.

There are some desired data quality constraints for the dataset:
1. All alphabetical characters in all columns should be capitalized.
2. Address data should be compatible to the standard in https://tools.usps.com/go/ZipLookupAction!input.action
3. The State column should contain the correct two character US state code.
4. City column should contain real city names.
5. ZIP column should be formatted as a 5 digit value.
6. SSN column should contain an 8-10 digit value.
Note that, you do not need to fix the DOB, POBox, and POCityStateZip columns.

In [173]:
df = pd.read_csv("./inputDB.csv")
print df.shape
f = open('us_cities.txt', 'r')
c = f.readlines()
cities = [str(city).rstrip().upper() for city in c]
reZIP = re.compile('[A-Z a-z]*(\d{5})')
reSSN = re.compile('^(\d{3}-\d{2}-\d{4})|(\d{3}\d{2}\d{4})|(\d{3}\d{3}\d{3})|\d{8}|\d{9}|\d{10}$')

(94306, 12)


In [174]:
#Rule 4 5 6 What if the City,SSN or ZIP is NaN ? Is this an error ?
#df['City'] =df['City'].fillna('ERROR')
#df['SSN'] =df['SSN'].fillna('ERROR')
#df['ZIP'] =df['ZIP'].fillna('ERROR')
reSSN.match("23334245")

In [197]:
states_list = ( 'AL', 'AK', 'AS', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FM', 'FL', 'GA', 'GU', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MH', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'MP', 'OH', 'OK', 'OR', 'PW', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VI', 'VA', 'WA', 'WV', 'WI', 'WY' )
df = pd.read_csv("./inputDB.csv")

is_capitalized = lambda x: x if str(x).upper() in str(x) else "ERROR"
is_properStateCode = lambda x: x if str(x).upper()  in states_list else "ERROR"
is_properCity = lambda x: x if str(x).upper()  in cities else "ERROR"
is_properZIP = lambda x: x if  reZIP.match(str(x)) else "ERROR"
is_properSSN = lambda x: x if  reSSN.match(str(x)) else "ERROR"

#Rule 1
df['FirstName'] = df['FirstName'].map(is_capitalized)
df['MiddleName'] = df['MiddleName'].map(is_capitalized)
df['LastName'] = df['LastName'].map(is_capitalized)
df['Address'] = df['Address'].map(is_capitalized)


#Rule 4 5 6
df['SSN'] = df['SSN'].map(is_properSSN)
df['City'] = df['City'].map(is_properCity).map(is_capitalized)
df['ZIP'] = df['ZIP'].map(is_properZIP)

# Rule 3/1
df['State'] = df['State'].map(is_properStateCode).map(is_capitalized).fillna('ERROR')




test = df.to_csv(index=False)
df.sample(100)

,RecID,FirstName,MiddleName,LastName,Address,City,State,ZIP,POBox,POCityStateZip,SSN,DOB
6492,A911186,ERROR,ERROR,ERROR,3550 CARTEQR DR APT 24,ERROR,ERROR,ERROR,PO BOX 5093,"SOUTH SAN FRANCISCO, CALIF.94083",199-04-3981,NaN
47666,A952360,IMCHAEL,L,HERNADNEZ,ERROR,ERROR,ERROR,Ca 95726,NaN,NaN,ERROR,NaN
52319,A957013,R,"IVLTRAKIS,",JR,ERROR,ERROR,TX,75670,PO BOX 1911,"MARSHALL, TX 75671",059-12-2644,NaN
90093,A994787,FRANCESCA,E,DIEZI,ERROR,ERROR,ERROR,93309,NaN,NaN,120406104,NaN
16681,A921375,ERROR,A,ERROR,ERROR,ERROR,ERROR,ERROR,NaN,NaN,ERROR,04-08-26
55598,A960292,J,K,ERROR,5114 LOETSCHER LANE,ERROR,ERROR,AR 72209,NaN,NaN,12404026,07-03-04
1397,A906091,BONNIE,C,CATO,ERROR,ERROR,ERROR,FL 32714,NaN,NaN,110741064,NaN
73147,A977841,SUK,ERROR,AHRT,ERROR,ERROR,ERROR,Myers Florida 33917,NaN,NaN,141503671,NaN
11960,A916654,ERROR,ERROR,ERROR,ERROR,ERROR,TX,75440,NaN,NaN,193931189,NaN
52591,A957285,ERROR,ERROR,ORDOG,228 BRANCO AVNUE,ERROR,ERROR,95301,Post Box 236,"ATWATER, CA95301",ERROR,NaN


### -----------------Error Detection Results-----------------
Number of detected cells: *374976* (Number of changed values)

Number of Correctly Detected cells: **302208** (cell was correctly identified as an error)

Detection precision: **0.805939580133** 

Detection recall: **0.778633742651** 

Detection F1: **0.792051390247 **

In [198]:
#Run this cell to ask webserver to evaluate the result
evaluate(test)

Number of dirty cells: 388126 (Number of erroneous cells in the data)
-----------------Error Detection Results-----------------
Number of detected cells: 374976 (Number of changed values)
Number of Correctly Detected cells: 302208 (cell was correctly identified as an error)
Detection precision: 0.805939580133 (ratio of correctly detected cells over all detected cells)
Detection recall: 0.778633742651 (ratio of correctly detected cells over all erroneous cells in the data)
Detection F1: 0.792051390247
-----------------Error Correction Results-----------------
Destroyed clean cells: 72768 (cell was correct but has been transformed into a wrong value)
Wrongly cleaned cells: 302208 (cell was wrong but the cleaning was also not correct)
Undetected cells: 85918 (cell was erroneous but was not touched)
Number of cells that need yet to be cleaned: 460894 (sum of the 3 cell types above)
Correction precision = 0.0 (ratio of correctly corrected cells over all changed cells)
Correction recall = 0.

u'Number of dirty cells: 388126 (Number of erroneous cells in the data)\n-----------------Error Detection Results-----------------\nNumber of detected cells: 374976 (Number of changed values)\nNumber of Correctly Detected cells: 302208 (cell was correctly identified as an error)\nDetection precision: 0.805939580133 (ratio of correctly detected cells over all detected cells)\nDetection recall: 0.778633742651 (ratio of correctly detected cells over all erroneous cells in the data)\nDetection F1: 0.792051390247\n-----------------Error Correction Results-----------------\nDestroyed clean cells: 72768 (cell was correct but has been transformed into a wrong value)\nWrongly cleaned cells: 302208 (cell was wrong but the cleaning was also not correct)\nUndetected cells: 85918 (cell was erroneous but was not touched)\nNumber of cells that need yet to be cleaned: 460894 (sum of the 3 cell types above)\nCorrection precision = 0.0 (ratio of correctly corrected cells over all changed cells)\nCorrect

In [193]:
print(RESULTS)

[[u'Detection precision: 0.805939580133 (ratio of correctly detected cells over all detected cells)', u'Detection recall: 0.778633742651 (ratio of correctly detected cells over all erroneous cells in the data)', u'Detection F1: 0.792051390247'], [u'Detection precision: 0.687125542205 (ratio of correctly detected cells over all detected cells)', u'Detection recall: 0.809342842273 (ratio of correctly detected cells over all erroneous cells in the data)', u'Detection F1: 0.743243419099']]
